In [2]:
# 경고 무시
import warnings
warnings.simplefilter('ignore')

# 자주 사용하는 패키지를 임포트
import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import sklearn as sk
import pgmpy
import scipy.stats

# matplotlib 설정
mpl.use('Agg')

# seaborn 설정
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

%matplotlib inline

In [3]:
- 1/4 * np.log2(1/2) - 1/4 * np.log2(1/4) - 1/4 * np.log2(1/8) - 1/4 * np.log2(1/8) 

2.25

In [4]:
from sklearn.datasets import load_iris

iris = load_iris()
idx = np.in1d(iris.target, [0, 1])
X = iris.data[idx, :]
y = iris.target[idx]
df = pd.DataFrame(X, columns=iris.feature_names)
df["y"] = iris.target[idx]
df["y_hat"] = (df["sepal length (cm)"] > 5.4).astype(int)
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),y,y_hat
95,5.7,3.0,4.2,1.2,1,1
96,5.7,2.9,4.2,1.3,1,1
97,6.2,2.9,4.3,1.3,1,1
98,5.1,2.5,3.0,1.1,1,0
99,5.7,2.8,4.1,1.3,1,1


In [5]:
from sklearn.metrics import log_loss

log_loss(df["y"], df["y_hat"])

3.799305383311686

### 쿨백-라이블러 발산
- 'p,q가 얼마나 닮았는가?' 의 척도
- 거리의 개념은 아니다.
- 기준값이 앞에 온다.
- 완전히 같은 분포를하면 0

### 가변길이 인코딩과 쿨백-라이블러 발산

In [6]:
# 4개의 글자 A, B, C, D로 씌여진 다음과 같은 문서를 가변길이 인코딩하는 경우를 생각하자.
N = 200
p = [1/2, 1/4, 1/8, 1/8]
doc0 = list("".join([int(N * p[i]) * c for i, c in enumerate("ABCD")]))
np.random.shuffle(doc0)
doc = "".join(doc0)
doc

'ABBBADDBAABDADABABAAACCABAABAACAABAAAAAADBAAABDABABAACBACBBAABBCAABAAAAAAAACABDDABBAACAACABADBBBCDBAACABBAAADBAAABADAAAABCAACBAADABADAACADBCDACBDAABAAACAABDBDABAABBCACDAABCDDDBBACBAAAACBACAACAACDABDAB'

In [7]:
from collections import Counter

p = np.array(list(Counter(doc).values())) / len(doc)
p

array([0.5  , 0.25 , 0.125, 0.125])

In [8]:
# 한글자당 인코딩된 글자수는 분포  q 의 엔트로피인 1.75가 된다.
sp.stats.entropy([1/2, 1/4, 1/8, 1/8], base=2)

1.75

In [ ]:
# 실제로 인코딩된 글자수에서 확인할 수 있다.

In [9]:
vl_encoder = {"A": "0", "B": "10", "C": "110", "D": "111"}
vl_encoded_doc = "".join([vl_encoder[c] for c in doc])
len(vl_encoded_doc) / len(doc)

1.75

In [ ]:
# 실제로 한글자당 인코딩된 글자수는 다음과 같이 계산할 수 있다.

In [10]:
encoder = {"A": "00", "B": "01", "C": "10", "D": "11"}
encoded_doc = "".join([encoder[c] for c in doc])
len(encoded_doc) / len(doc)

2.0

- entropy 함수는 두 개의 확률분포를 인수로 넣으면 쿨백-라이블러 발산을 계산해준다.
- base 인수를 2로 설정하는 것을 잊지 말아야 한다.

In [11]:
sp.stats.entropy([1/2, 1/4, 1/8, 1/8], [1/4, 1/4, 1/4, 1/4], base=2)

0.24999999999999997